Prueba con DeepSORT Realtime

In [ ]:
def getColor(class_name):
    switch = {
        'player': (255, 0, 0),       # Azul
        'basketball': (0, 165, 255), # Naranja
        'rim': (0, 0, 255),          # Rojo
        'made-shot': (0, 255, 0)     # Verde
    }
    return switch.get(class_name, (0, 0, 0)) 

def drawBBox(frame, x1, y1, x2, y2, label, class_name):
    color = getColor(class_name)
    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
    cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

def drawPosition(frame, position, position_label):
    cv2.ellipse(frame, (int(position[0]), int(position[1])), (9, 3), 0, 0, 360, (0, 0, 255), -1)
    cv2.putText(frame, position_label, (int(position[0]) - 50, int(position[1]) + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

In [ ]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort


object_detector = YOLO("../object_detection/runs/detect/bod_v1/weights/best.pt")

tracker = DeepSort(max_age=50, n_init=5, nms_max_overlap=1.0)

video_path = "../clips/ClipLF1.mp4"
output_path = "../output/ClipLF1_tracking_output.mp4"

cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*"avc1")
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

MAX_PLAYERS = 10

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    object_detection_results = object_detector(frame)

    detections = []
    # Análisis de las detecciones de jugadores
    for result in object_detection_results[0].boxes.data.tolist():  # Obtener los resultados como lista
        x1, y1, x2, y2, conf, cls = result  # Coordenadas, confianza y clase
        cls = int(cls)

        # Filtrar solo por las clases deseadas
        if object_detector.names[cls] in ['player'] and conf > 0.5:
            detections.append(([x1, y1, x2-x1, y2-y1], conf, cls))

    tracking_results = tracker.update_tracks(detections, frame=frame)

    # Filtrar solo los tracks confirmados
    confirmed_tracks = [track for track in tracking_results if track.is_confirmed()]

    # Ordenar por track_id (IDs más bajos primero) y mantener solo 10
    if len(confirmed_tracks) > MAX_PLAYERS:
        confirmed_tracks = sorted(confirmed_tracks, key=lambda t: t.track_id)[:MAX_PLAYERS]

    for track in confirmed_tracks:
        track_id = track.track_id  # Obtener el ID del track
        x1, y1, x2, y2 = track.to_ltrb()  # Bounding box en formato [left, top, right, bottom]
        drawBBox(frame, x1, y1, x2, y2, f"ID: {track_id}", 'player')

        position = (int((x1 + x2) / 2), y2)
        position_label = f"x:{int(position[0])} y:{int(position[1])}"
        drawPosition(frame, position, position_label)



    # Mostrar el frame procesado en pantalla
    cv2.imshow('Resultados', frame)

    # Esperar por una tecla: espacio para avanzar, 'q' para salir
    key = cv2.waitKey(0) & 0xFF  # Espera indefinidamente hasta que se presione una tecla
    if key == ord('q'):
        break
    elif key == ord(' '):  # Espacio para continuar
        # Escribir el frame procesado en el video de salida
        out.write(frame)


# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video procesado guardado en {output_path}")


Tracking V2

In [ ]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort


object_detector = YOLO("../object_detection/runs/detect/bod_v1/weights/best.pt")

tracker = DeepSort(max_age=50, n_init=5, nms_max_overlap=1.0)

video_path = "../clips/ClipS1.mp4"
output_path = "../output/ClipS1_tracking_output.mp4"

cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*"avc1")
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

MAX_PLAYERS = 10

MAX_FRAMES_WITHOUT_DETECTION = 3

# Diccionario para mapear track_id originales a IDs fijos (1-10)
id_mapping = {}
lost_ids = set(range(1, MAX_PLAYERS + 1))  # IDs disponibles (1-10)
active_ids = set()  # IDs actualmente asignados

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    object_detection_results = object_detector(frame)

    detections = []
    # Análisis de las detecciones de jugadores
    for result in object_detection_results[0].boxes.data.tolist():  # Obtener los resultados como lista
        x1, y1, x2, y2, conf, cls = result  # Coordenadas, confianza y clase
        cls = int(cls)

        # Filtrar solo por las clases deseadas
        if object_detector.names[cls] in ['player'] and conf > 0.5:
            detections.append(([x1, y1, x2 - x1, y2 - y1], conf, cls))

    tracking_results = tracker.update_tracks(detections, frame=frame)

    # Filtrar solo los tracks confirmados
    confirmed_tracks = [track for track in tracking_results 
                        if track.is_confirmed()
                        and track.time_since_update <= MAX_FRAMES_WITHOUT_DETECTION]

    for track in confirmed_tracks:
        original_id = track.track_id  # ID asignado por DeepSORT

        if original_id in id_mapping:
            fixed_id = id_mapping[original_id]  # Usar el ID ya asignado
        elif lost_ids:
            fixed_id = lost_ids.pop()  # Tomar un ID libre
            id_mapping[original_id] = fixed_id  # Asignarlo al nuevo jugador
            active_ids.add(fixed_id)  # Registrar el ID como activo
        else:
            continue  # Si no hay IDs disponibles, ignoramos la detección

        x1, y1, x2, y2 = track.to_ltrb()  # Bounding box en formato [left, top, right, bottom]
        drawBBox(frame, x1, y1, x2, y2, f"ID: {fixed_id}", 'player')

        position = (int((x1 + x2) / 2), y2)
        position_label = f"x:{int(position[0])} y:{int(position[1])}"
        drawPosition(frame, position, position_label)

    # Identificar jugadores que han desaparecido
    active_now = {id_mapping[track.track_id] for track in confirmed_tracks if track.track_id in id_mapping}
    lost_ids.update(active_ids - active_now)  # Recuperar IDs de jugadores perdidos
    active_ids = active_now  # Actualizar IDs activos

    # Mostrar el frame procesado en pantalla
    cv2.imshow('Resultados', frame)

    # Esperar por una tecla: espacio para avanzar, 'q' para salir
    key = cv2.waitKey(0) & 0xFF  # Espera indefinidamente hasta que se presione una tecla
    if key == ord('q'):
        break
    elif key == ord(' '):  # Espacio para continuar
        # Escribir el frame procesado en el video de salida
        out.write(frame)


# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video procesado guardado en {output_path}")
